In [1]:
!pip install -qU langchain langchain_cohere

**Option one**

In [2]:
from langchain.tools import tool

@tool
def add(a: int, b: int) -> int:
    """Adds a and b"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b"""
    return a * b

tools = [add, multiply]

**Option two**

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Add(BaseModel):
    """Add two integers together"""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class Multiply(BaseModel):
    """Multiply two integers together"""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

tools = [Add, Multiply]

In [6]:
import os
import getpass
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API key:")

Cohere API key: ········


In [7]:
from langchain_cohere import ChatCohere

llm = ChatCohere(mode="command-r", temperature=0)

In [8]:
llm.invoke("When Cohere was founded?")

AIMessage(content='Cohere was founded in 2019 by Aidan Gomez, Nick Frosst, and Ivan Zhang. The company is headquartered in Toronto, Canada, and has since expanded its operations to several other locations. Cohere has gained recognition for its innovative large language models and natural language processing capabilities, offering advanced AI solutions to a diverse range of industries and use cases.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'dc13fc95-739c-4201-a6d0-aa94ab444d52'}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'dc13fc95-739c-4201-a6d0-aa94ab444d52'}, id='run-a1e52adc-0197-4318-8e73-35093a57adbb-0')

In [9]:
llm_with_tools = llm.bind_tools(tools)

In [10]:
llm_with_tools.invoke("what is 3*2?")

AIMessage(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '2f6b9471-c39b-4944-a27f-7df1e7ad06ba', 'tool_calls': [{'id': '2f6b9471-c39b-4944-a27f-7df1e7ad06ba', 'function': {'name': 'Multiply', 'arguments': '{"a": 3, "b": 2}'}, 'type': 'function'}]}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '2f6b9471-c39b-4944-a27f-7df1e7ad06ba', 'tool_calls': [{'id': '2f6b9471-c39b-4944-a27f-7df1e7ad06ba', 'function': {'name': 'Multiply', 'arguments': '{"a": 3, "b": 2}'}, 'type': 'function'}]}, id='run-449e0181-2f3c-4f2e-ac76-b5bad449b17d-0', tool_calls=[{'name': 'Multiply', 'args': {'a': 3, 'b': 2}, 'id': None}])

In [11]:
llm_with_tools.invoke("What is 5-2?")

AIMessage(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '0f32f47b-c2d8-404c-8092-63f3834a6f8b'}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '0f32f47b-c2d8-404c-8092-63f3834a6f8b'}, id='run-a59b4270-8a3e-4c09-8baa-a17beb0e541a-0')

In [12]:
llm_with_tools.invoke("What is 5+2*3?")

AIMessage(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '9ecc9e2a-2af5-4dca-bc28-fb6179f576d4', 'tool_calls': [{'id': '9ecc9e2a-2af5-4dca-bc28-fb6179f576d4', 'function': {'name': 'Multiply', 'arguments': '{"a": 2, "b": 3}'}, 'type': 'function'}, {'id': '9ecc9e2a-2af5-4dca-bc28-fb6179f576d4', 'function': {'name': 'Add', 'arguments': '{"a": 5, "b": 6}'}, 'type': 'function'}]}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '9ecc9e2a-2af5-4dca-bc28-fb6179f576d4', 'tool_calls': [{'id': '9ecc9e2a-2af5-4dca-bc28-fb6179f576d4', 'function': {'name': 'Multiply', 'arguments': '{"a": 2, "b": 3}'}, 'type': 'function'}, {'id': '9ecc9e2a-2af5-4dca-bc28-fb6179f576d4', 'function': {'name': 'Add', 'arguments': '{"a": 5, "b": 6}'}, 'type': 'function'}]}, id='run-cb651069-913f-4e9e-9439-8d

**Use output parser**

In [15]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[Multiply, Add])
chain.invoke("what is 5+2*3?")

[Multiply(a=2, b=3), Add(a=5, b=6)]

**Streaming**

In [32]:
async for chunk in llm_with_tools.astream("what is 5+2? and what is 10*3?"):
    print(chunk.tool_call_chunks)

[{'name': 'Add', 'args': '{"a": 5, "b": 2}', 'id': 'b737a453-3a12-4f50-9728-9e58fcb2627b', 'index': None}, {'name': 'Multiply', 'args': '{"a": 10, "b": 3}', 'id': 'b737a453-3a12-4f50-9728-9e58fcb2627b', 'index': None}]


In [30]:
first = True
async for chunk in llm_with_tools.astream("what is 5+2*3?"):
    if first:
        gathered = chunk,
        first = False
    else:
        gathered += chunk
    print(gathered)

(AIMessageChunk(content='', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'f8897602-935d-48a9-996d-bfeeb30da37b', 'tool_calls': [{'id': 'f8897602-935d-48a9-996d-bfeeb30da37b', 'function': {'name': 'Multiply', 'arguments': '{"a": 2, "b": 3}'}, 'type': 'function'}, {'id': 'f8897602-935d-48a9-996d-bfeeb30da37b', 'function': {'name': 'Add', 'arguments': '{"a": 5, "b": 6}'}, 'type': 'function'}]}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'f8897602-935d-48a9-996d-bfeeb30da37b', 'tool_calls': [{'id': 'f8897602-935d-48a9-996d-bfeeb30da37b', 'function': {'name': 'Multiply', 'arguments': '{"a": 2, "b": 3}'}, 'type': 'function'}, {'id': 'f8897602-935d-48a9-996d-bfeeb30da37b', 'function': {'name': 'Add', 'arguments': '{"a": 5, "b": 6}'}, 'type': 'function'}]}, id='run-125bfc9a-c4a4-4d32-8